地名拼写错误、简称混用、历史名称不一致……地理行业的数据清洗痛点之一就是“名称对不齐”。今天我们分享一种智能方式：用 Embedding 向量模型自动对齐地名，打造更聪明的数据查询系统。

📌 地理行业常见问题场景

在做地理大数据分析或问答接口时，你一定遇到过：
	•	用户输入“成都高新区”，但你数据库里叫“高新技术产业开发区（成都）”；
	•	用户写了“渝北区”，但你表里是“重庆市渝北区”；
	•	数据源来自多系统，有“中山”、“中山市”、“中山（广东）”三种叫法。

这些拼写、格式、简称差异，导致无法正确匹配目标记录。传统 string matching（字符串匹配）方法太死板。怎么办？

💡 解决方案：用 Embedding 模型做智能相似度匹配

我们将所有地名编码为向量，再通过向量检索寻找“最相似”的标准名称，从而实现：

用户输入 → 向量查询 → 返回标准地名 → 精准过滤

✅ 第一步：构建基本环境和准备模型

我们首先需要引入必要的组件，并初始化一个大语言模型（本文使用阿里 Qwen）、一个中文文本嵌入模型（DashScope Embedding）以及一个简单的向量数据库（这里用内存实现）。

In [23]:
from config import *  # 包含密钥配置
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

📌 第二步：构建中文地名数据库并进行向量化

我们使用一个简单的地名列表模拟真实业务中的地理名称标准库。

In [2]:
llm = ChatOpenAI(model="qwen-max")
embeddings = DashScopeEmbeddings(model="text-embedding-v3")
vector_store = InMemoryVectorStore(embeddings)

In [24]:
geo_names = [
    "北京市海淀区", "北京市朝阳区", "上海市浦东新区", "广东省深圳市南山区",
    "成都市高新区", "重庆市渝北区", "杭州市西湖区", "中山市",
    "武汉市洪山区", "西安市雁塔区"
]
_ = vector_store.add_texts(geo_names)

每个地名会被转换为一个多维向量，从而可以通过语义进行模糊比对，不再依赖于完全一致的关键词。

🔍 第三步：查询相似地名（Embedding 检索）

现在我们尝试用用户的输入进行相似度查询。以下是一个最基础的检索示例：

In [35]:
query = "成都高新区"
results = vector_store.similarity_search_with_score(query, k=3)

for name, score in results:
    print(f"候选名称：{name.page_content} | 相似度得分：{score:.4f}")

候选名称：成都市高新区 | 相似度得分：0.9509
候选名称：成都市高新区 | 相似度得分：0.9509
候选名称：上海市浦东新区 | 相似度得分：0.6413


可以看到，即使“成都高新区”与“成都市高新区”不完全一致，Embedding 仍然能识别出它们在语义上非常相近。

🛠 第四步：封装为工具组件（可用于 Agent）

我们可以把这个向量检索过程封装为一个工具，未来可以集成到 LangChain Agent 框架中，让大模型自主调用：

In [8]:
from langchain.agents.agent_toolkits import create_retriever_tool

retriever = vector_store.as_retriever(search_kwargs={"k": 3})
retriever_tool = create_retriever_tool(
    retriever,
    name="search_geo_names",
    description="用于查找最相似的地理名称，如城市、区县、开发区等"
)

🧠 第五步：引入大模型进行最终智能判断

虽然 Embedding 可以找出语义相近的候选地名，但我们仍然需要：

•	判断相似度是否“足够高”才能接受

•	在多个候选地名中做出更合理的推断

•	将判断逻辑结构化输出（便于下游业务使用）

因此我们引入大语言模型，并用 Pydantic 定义结构化输出格式：

In [36]:
from typing import Literal
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda


class MatchResult(BaseModel):
    """地名匹配结果"""
    name: str = Field(..., description="最匹配的地名")
    score: float = Field(..., description="地名匹配的相似度分数")
    accepted: Literal[True, False] = Field(..., description="该地名是否可以被接受")

接着我们构造一个智能判断的 Prompt，告诉大模型当前有哪些候选地名，它们和原始输入之间的相似度是多少，请它给出最终判断结果：

In [37]:
parser = PydanticOutputParser(pydantic_object=MatchResult)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你是一个中文地名智能判断助手。请根据原始查询和候选地名相似度，返回匹配结果。\n{format_instructions}"
        ),
        ("human", "{query}")
    ]
).partial(format_instructions=parser.get_format_instructions())

🔗 第六步：拼接 Chain，实现自动判断

In [38]:
def build_prompt_input(query: str, top_k: int = 3) -> dict:
    results = vector_store.similarity_search_with_score(query, k=top_k)
    formatted = "\n".join([
        f"- {doc.page_content}（相似度: {score:.4f}）" for doc, score in results
    ])
    prompt_text = \
        f"""原始查询: {query}
            候选地名如下：
            {formatted}
        """
    return {"query": prompt_text}


# 拼接完整处理链条
chain = (
        RunnableLambda(build_prompt_input)
        | prompt
        | llm
        | parser
)

🔬 第七步：运行一个完整查询

In [40]:
query = "难山区"
chain.invoke(query)

MatchResult(name='广东省深圳市南山区', score=0.6222, accepted=True)